<a href="https://colab.research.google.com/github/sanjayy0612/stock_predictor-NVDIA-/blob/main/NVDIA_STOCK_PREDICTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance


In [2]:
import yfinance as yf
import pandas as pd


nvda = yf.download("NVDA", start="2022-01-01", end="2025-08-28", interval='1d')

/tmp/ipython-input-4101073138.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nvda = yf.download("NVDA", start="2022-01-01", end="2025-08-28", interval='1d')
[*********************100%***********************]  1 of 1 completed


In [3]:

if isinstance(nvda.columns, pd.MultiIndex):
    nvda.columns = [col[0] for col in nvda.columns]

nvda["Return"] = nvda["Close"].pct_change()
nvda["MA10"] = nvda["Close"].rolling(window=10).mean()
nvda["MA50"] = nvda["Close"].rolling(window=50).mean()
nvda["Volatility"] = nvda["Return"].rolling(window=10).std()
nvda = nvda.dropna()
print(nvda.head(15))
print("\nAvailable columns:", nvda.columns)


                Close       High        Low       Open     Volume    Return  \
Date                                                                          
2022-03-15  22.935059  22.999952  21.286786  21.482462  491996000  0.077028   
2022-03-16  24.455544  24.556377  23.133731  23.461188  671422000  0.066295   
2022-03-17  24.725100  24.800972  23.866519  24.080167  471941000  0.011022   
2022-03-18  26.409315  26.525125  24.583337  24.758047  730719000  0.068118   
2022-03-21  26.689854  27.107165  25.924120  26.463229  591727000  0.010623   
2022-03-22  26.480200  27.193021  26.028947  26.682864  547007000 -0.007855   
2022-03-23  25.591667  26.568051  25.532765  26.082853  502120000 -0.033555   
2022-03-24  28.103512  28.273231  25.864216  26.122789  877379000  0.098151   
2022-03-25  27.646269  28.311169  27.225965  27.839949  579016000 -0.016270   
2022-03-28  28.172396  28.203345  27.161068  27.710161  425494000  0.019031   
2022-03-29  28.608677  28.898197  27.933793  28.6476

In [4]:
nvda["Target_UpDown"] = (nvda["Close"].shift(-1) > nvda["Close"]).astype(int)
nvda["Target_Close"] = nvda["Close"].shift(-1)
nvda.dropna()

,Close,High,Low,Open,Volume,Return,MA10,MA50,Volatility,Target_UpDown,Target_Close
Date,,,,,,,,,,,
2022-03-15,22.935059,22.999952,21.286786,21.482462,491996000,0.077028,22.543809,24.608723,0.047368,1,24.455544
2022-03-16,24.455544,24.556377,23.133731,23.461188,671422000,0.066295,22.571363,24.496511,0.051139,1,24.725100
2022-03-17,24.725100,24.800972,23.866519,24.080167,471941000,0.011022,22.676390,24.406280,0.050523,1,26.409315
2022-03-18,26.409315,26.525125,24.583337,24.758047,730719000,0.068118,23.027509,24.383392,0.052080,1,26.689854
2022-03-21,26.689854,27.107165,25.924120,26.463229,591727000,0.010623,23.564820,24.354656,0.042995,0,26.480200
...,...,...,...,...,...,...,...,...,...,...,...
2025-08-20,175.399994,176.000000,168.800003,175.169998,215142700,-0.001366,180.579999,165.672800,0.013444,0,174.979996
2025-08-21,174.979996,176.899994,173.809998,174.850006,140040900,-0.002395,180.000998,166.293400,0.013008,1,177.990005
2025-08-22,177.990005,178.589996,171.199997,172.610001,172789400,0.017202,179.529999,166.996600,0.013912,1,179.809998


In [5]:
from sklearn.model_selection import train_test_split

X = nvda[["Return", "MA10", "MA50", "Volatility"]]
y = nvda[["Target_UpDown", "Target_Close"]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

y_class_train = y_train["Target_UpDown"]
y_class_test  = y_test["Target_UpDown"]

y_reg_train   = y_train["Target_Close"]
y_reg_test    = y_test["Target_Close"]